# Paper Data Processing

## Mesh to SDF

In [ ]:
import os
from pathlib import Path
import trimesh
import comptools
import torch
import numpy as np
import trimesh
from alphashape import alphashape


# The path to the mesh file
input_data_folder_path = r"/home/student/Documents/Data/Input_puntenwolken/Scanned data"
saveFolder = r""

voxelGridSize = 64

input_list = []
for subdir, dirs, files in os.walk(input_data_folder_path):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".obj"):
            #print (filepath)
            input_list.append(filepath)

print(input_list)

[]


In [ ]:
import trimesh.transformations as transformations
import math

for i in range(len(input_list)):
    input_path = input_list[i]
    # Get the shape completion input
    fileFolder = Path(input_path).parent.name
    fileName = Path(input_path).stem
    print(fileFolder,fileName)
    savePath = os.path.join(saveFolder, fileFolder, fileName)
    comptools.make_dir_if_not_exist(savePath)

    # import mesh
    mesh = trimesh.load(input_path)

    # Creating a alpha shape to create a more complex hull
    alphaValue = 8
    alphaMesh = alphashape(mesh.vertices, alphaValue)
    alphaMesh.invert()
    mesh = alphaMesh

    # Recenter the mesh to the origin
    mesh.apply_translation(-mesh.bounding_box.centroid)

    # Normalize the scale of the mesh
    # Scale so the bounding box fits within a unit cube
    bounding_box_extents = mesh.bounding_box.extents
    scale_factor = 1.0 / max(bounding_box_extents)
    mesh.apply_scale(scale_factor)

    #angle = math.pi
    #direction = [0, 0, 1]
    #angle2 = math.pi/4
    #direction2 = [0, 1, 0]
    #center = [0, 0, 0]

    #rot_matrix = transformations.rotation_matrix(angle, direction, center)
    #mesh.apply_transform(rot_matrix)
    #rot_matrix2 = transformations.rotation_matrix(angle2, direction2, center)
    #mesh.apply_transform(rot_matrix2)
    print("mesh normalised")

    #SDF creation
    sdf,sdfMesh = comptools.mesh_to_sdf_tensor(comptools.as_mesh(mesh), voxelGridSize, recenter=True)

    fixedMeshPath = os.path.join(savePath,"sdf.obj")
    sdfPath = fixedMeshPath[:-4] + ".npy"
    tfPath = fixedMeshPath[:-4] + ".pt"
    print("mesh created")

    # output
    mesh.export(os.path.join(savePath, "gt.obj"))
    sdfMesh.export(fixedMeshPath)
    np.save(sdfPath, sdf)
    # Tensor
    shapedData = np.reshape(sdf, [1, 1, 64, 64, 64])
    torch.save(torch.from_numpy(shapedData[:,:,:64,:64,:64]), tfPath)



buis_recht recht_versmal_lang
mesh normalised


KeyboardInterrupt: 